In [1]:
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from pathlib import Path
from pysam import VariantFile
import seaborn as sns
from vcf_to_df import vcf_to_df
from vcf_to_df import vcf_to_df_simple

mpl.rcParams['figure.dpi']= 200

%load_ext autoreload
%autoreload 2

## Filter VCFs for region of interest. In this case intersection of AIO and Exome.

In [2]:
%%bash

bed=~/XTHS-analysis/data/bed/AIO_EXOME_INTERSECT.hg19.bed
vcf_dir=~/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect

lst='FRFZ FFPE_AIO_100_S FFPE_AIO_3_A1 FFPE_AIO_3_A2 FFPE_AIO_3_A3 FFPE_AIO_3_A4 FFPE_AIO_3_B1 FFPE_AIO_3_B2 FFPE_AIO_3_C1 FFPE_AIO_3_C2 FFPE_AIO_3_S'

for i in $lst ; 
    do bgzip -c -d ${vcf_dir}/${i}-ensemble-annotated.vcf.gz | \
    bedtools intersect -header -a stdin -b $bed | \
    bgzip -c > ${vcf_dir}/${i}-ensemble.region.vcf.gz; 
done

# Filter VCF for benign in clinvar, non-COSMIC, frequent in population variants

In [3]:
samples = ['FRFZ',
           'FFPE_AIO_3_A1',
           'FFPE_AIO_3_A2',
           'FFPE_AIO_3_A3',
           'FFPE_AIO_3_A4',
           'FFPE_AIO_3_B1',
           'FFPE_AIO_3_B2',
           'FFPE_AIO_3_C1',
           'FFPE_AIO_3_C2',
           'FFPE_AIO_3_S',
           'FFPE_AIO_100_S']

In [4]:
not_path_in_clinvar = ['benign',' not_provided','uncertain','drug_response']

for sample in samples:
    
    filename = '/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/{}-ensemble.region.vcf.gz'.format(sample)
    path = Path(filename)
    
    out_file = open(str(path.parent) 
                    + '/' 
                    + str(path.stem.split('-')[0]) 
                    + '-ensemble.region.gmfilter.vcf',
                    mode='w')
    
    
    vcf_in = VariantFile(path)
    out_file.write(vcf_in.header.__str__())
    
    for rec in vcf_in:
        
        info_dct = rec.info
                   
        if 'EPR' in info_dct.keys():

            # Variant not found in database
            if info_dct['EPR'][0] == 'pass':
                out_file.write(rec.__str__())
                continue

            # Variant found in COSMIC database
            elif 'cosmic_ids' in info_dct.keys():
                out_file.write(rec.__str__())
                continue

            # Variant found at max aaf > 0.01 and not deemed pathogenic in clinvar
            else:
                try:
                    if isinstance(info_dct['max_aaf_all'], float):
                        max_aaf_all = float(info_dct['max_aaf_all'])
                    else:
                        max_aaf_all = float(info_dct['max_aaf_all'][0])
                except:
                    print('SOMETHING WRONG, NO MAX AAF')
                    print(info_dct['max_aaf_all'])
                    out_file.write(rec.__str__())
                    continue

                if max_aaf_all > 0.001:
                    if 'clinvar_sig' in info_dct.keys():
                        s = "".join(info_dct['clinvar_sig'])
                        path_label = s.split('=')[-1].lower()
                        if any(non_path in path_label for non_path in not_path_in_clinvar):
                            out_file.write(rec.__str__())
                        else:
                            continue
                    else:
                        continue
                else:
                    out_file.write(rec.__str__())
        else:
            out_file.write(rec.__str__())

    out_file.close()

## Filter for AF >= 0.1

In [5]:
%%bash

lst='FRFZ FFPE_AIO_3_A1 FFPE_AIO_3_A2 FFPE_AIO_3_A3 FFPE_AIO_3_A4 FFPE_AIO_3_B1 FFPE_AIO_3_B2 FFPE_AIO_3_C1 FFPE_AIO_3_C2 FFPE_AIO_3_S FFPE_AIO_100_S'

for i in $lst; do
    
    vcf_in_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.vcf
    vcf_out_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.0.1.vcf
    
    bcftools filter -i '(QUAL>50 && INFO/DP>5) && (INFO/AF>=0.1)' $vcf_in_path > $vcf_out_path
    bgzip -c ${vcf_out_path} > ${vcf_out_path}.gz
    tabix -p vcf ${vcf_out_path}.gz
done

## Filter for quality, depth > 5 and AF >= 0.1, gzip and index each VCF file

In [6]:
%%bash

lst='FRFZ FFPE_AIO_3_A1 FFPE_AIO_3_A2 FFPE_AIO_3_A3 FFPE_AIO_3_A4 FFPE_AIO_3_B1 FFPE_AIO_3_B2 FFPE_AIO_3_C1 FFPE_AIO_3_C2 FFPE_AIO_3_S FFPE_AIO_100_S'

for i in $lst; do
    
    vcf_in_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.vcf
    vcf_out_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.dp_qualfilter.vcf
    
    bcftools filter -i '(QUAL>50 && INFO/DP>5) && (INFO/AF>=0.1)' $vcf_in_path > $vcf_out_path
    bgzip -c ${vcf_out_path} > ${vcf_out_path}.gz
    tabix -p vcf ${vcf_out_path}.gz
done

## Filter for AF >= 0.2, gzip and index each VCF file

In [7]:
%%bash

lst='FRFZ FFPE_AIO_3_A1 FFPE_AIO_3_A2 FFPE_AIO_3_A3 FFPE_AIO_3_A4 FFPE_AIO_3_B1 FFPE_AIO_3_B2 FFPE_AIO_3_C1 FFPE_AIO_3_C2 FFPE_AIO_3_S FFPE_AIO_100_S'

for i in $lst; do
    
    vcf_in_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.dp_qualfilter.vcf
    vcf_out_path=/Users/DanielaNachmanson/XTHS-analysis/data/vcf_AIO/FF_vs_FFPE_AIO_Intersect/${i}-ensemble.region.gmfilter.dp_qualfilter.0.2.vcf
    
    bcftools filter -i '(INFO/AF>=0.2)' $vcf_in_path > $vcf_out_path
    bgzip -c ${vcf_out_path} > ${vcf_out_path}.gz
    tabix -p vcf ${vcf_out_path}.gz
done